## Deliverable 2. Create a Customer Travel Destinations Map.

In [30]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
!jupyter nbextension enable --py gmaps
import numpy as np
import citipy

# Import API key
from config import g_key

# Configure Google Maps API key
gmaps.configure(api_key=g_key)

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [31]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/davek/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Percent Humidity,Percent Cloudiness,Wind Speed,Current Weather Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,60.12,73,100,10.85,overcast clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,51.46,82,100,10.36,moderate rain
2,2,Kruisfontein,ZA,-34.0033,24.7314,61.02,86,0,1.45,clear sky
3,3,Bengkulu,ID,-3.8004,102.2655,74.05,87,100,4.50,overcast clouds
4,4,Qaanaaq,GL,77.4840,-69.3632,-23.55,56,0,7.43,clear sky


In [32]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter the minimum temperature you would prefer for your trip destination: "))
max_temp = float(input("Enter the maximum temperature you would prefer for your trip destination: "))

Enter the minimum temperature you would prefer for your trip destination: 70
Enter the maximum temperature you would prefer for your trip destination: 90


In [33]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Percent Humidity,Percent Cloudiness,Wind Speed,Current Weather Description
3,3,Bengkulu,ID,-3.8004,102.2655,74.05,87,100,4.50,overcast clouds
5,5,Sao Joao Da Barra,BR,-21.6403,-41.0511,73.18,93,100,9.37,overcast clouds
7,7,Luderitz,NaN,-26.6481,15.1594,75.56,65,100,10.16,overcast clouds
13,13,Vieux-Habitants,GP,16.0589,-61.7659,76.71,66,18,11.79,few clouds
15,15,Codajas,BR,-3.8367,-62.0569,75.11,94,95,2.68,overcast clouds
17,17,Carnarvon,AU,-24.8667,113.6333,78.87,73,40,3.44,scattered clouds
18,18,Puerto Ayora,EC,-0.7393,-90.3518,72.55,99,84,1.99,broken clouds
23,23,Araripina,BR,-7.5761,-40.4983,70.81,83,77,1.83,broken clouds
25,25,Tabou,CI,4.4230,-7.3528,78.55,88,51,5.26,broken clouds
26,26,Maragogi,BR,-9.0122,-35.2225,80.76,78,80,11.92,broken clouds


In [34]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                        0
City                           0
Country                        2
Lat                            0
Lng                            0
Max Temp                       0
Percent Humidity               0
Percent Cloudiness             0
Wind Speed                     0
Current Weather Description    0
dtype: int64

In [35]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                        0
City                           0
Country                        0
Lat                            0
Lng                            0
Max Temp                       0
Percent Humidity               0
Percent Cloudiness             0
Wind Speed                     0
Current Weather Description    0
dtype: int64

In [36]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather Description,Lat,Lng,Hotel Name
3,Bengkulu,ID,74.05,overcast clouds,-3.8004,102.2655,
5,Sao Joao Da Barra,BR,73.18,overcast clouds,-21.6403,-41.0511,
13,Vieux-Habitants,GP,76.71,few clouds,16.0589,-61.7659,
15,Codajas,BR,75.11,overcast clouds,-3.8367,-62.0569,
17,Carnarvon,AU,78.87,scattered clouds,-24.8667,113.6333,
18,Puerto Ayora,EC,72.55,broken clouds,-0.7393,-90.3518,
23,Araripina,BR,70.81,broken clouds,-7.5761,-40.4983,
25,Tabou,CI,78.55,broken clouds,4.4230,-7.3528,
26,Maragogi,BR,80.76,broken clouds,-9.0122,-35.2225,
28,Comodoro Rivadavia,AR,75.06,clear sky,-45.8667,-67.5000,


In [38]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
#7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Weather Description,Lat,Lng,Hotel Name
3,Bengkulu,ID,74.05,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
5,Sao Joao Da Barra,BR,73.18,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas
13,Vieux-Habitants,GP,76.71,few clouds,16.0589,-61.7659,Gîtes Dardanelle
15,Codajas,BR,75.11,overcast clouds,-3.8367,-62.0569,Atlântic Hotel
17,Carnarvon,AU,78.87,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon


In [40]:
# 8a. Create the output file
output_data_file = 'Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))